In [3]:
from transformers import pipeline
import pandas as pd
import torch
from tqdm import tqdm

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import random

In [5]:
tqdm.pandas()

In [6]:
classifier = pipeline('sentiment-analysis', device=0)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [7]:
df = pd.read_parquet("./700077347251945472.parquet")

In [8]:
def get_sentiment(x):
    try: 
        return classifier(x)
    except:
        return None

In [9]:
df["pipe_sentiment_analysis"] = df.content.progress_apply(get_sentiment)

  0% 2/492466 [00:00<41:33:55,  3.29it/s]/opt/conda/lib/python3.7/site-packages/transformers/pipelines/base.py:1046: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
 43% 211986/492466 [24:19<32:43, 142.83it/s] Token indices sequence length is longer than the specified maximum sequence length for this model (658 > 512). Running this sequence through the model will result in indexing errors
 64% 314488/492466 [36:01<19:46, 150.06it/s] IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

 89% 435971/492466 [49:49<06:30, 144.66it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to a

In [11]:
df.to_parquet('./700077347251945472_sentiment.parquet')

In [12]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from transformers import pipeline

In [13]:
tokenizer_loaded = RobertaTokenizer.from_pretrained('zhayunduo/roberta-base-stocktwits-finetuned')
model_loaded = RobertaForSequenceClassification.from_pretrained('zhayunduo/roberta-base-stocktwits-finetuned')

Downloading: 100% 899k/899k [00:00<00:00, 40.3MB/s]
Downloading: 100% 456k/456k [00:00<00:00, 35.7MB/s]
Downloading: 100% 772/772 [00:00<00:00, 663kB/s]
Downloading: 100% 1.16k/1.16k [00:00<00:00, 1.09MB/s]
Downloading: 100% 782/782 [00:00<00:00, 696kB/s]
Downloading: 100% 499M/499M [00:11<00:00, 43.9MB/s] 


In [19]:
stocktwits = pipeline("text-classification", model=model_loaded, tokenizer=tokenizer_loaded, device=2)

In [20]:
def get_sentiment_stocktwits(x):
    try: 
        return stocktwits(x)
    except:
        return None

In [ ]:
import emoji

# the model was trained upon below preprocessing
def process_text(texts):

  # remove URLs
  texts = re.sub(r'https?://\S+', "", texts)
  texts = re.sub(r'www.\S+', "", texts)
  # remove '
  texts = texts.replace('&#39;', "'")
  # remove symbol names
  texts = re.sub(r'(\#)(\S+)', r'hashtag_\2', texts)
  texts = re.sub(r'(\$)([A-Za-z]+)', r'cashtag_\2', texts)
  # remove usernames
  texts = re.sub(r'(\@)(\S+)', r'mention_\2', texts)
  # demojize
  texts = emoji.demojize(texts, delimiters=("", " "))

  return texts.strip()

In [ ]:
df["stocktwits"] = df.content.progress_apply(get_sentiment_stocktwits)

  0% 8/492466 [00:00<1:50:10, 74.50it/s]/opt/conda/lib/python3.7/site-packages/transformers/pipelines/base.py:1046: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
 43% 211985/492466 [43:06<57:28, 81.34it/s]  Token indices sequence length is longer than the specified maximum sequence length for this model (744 > 512). Running this sequence through the model will result in indexing errors
 87% 426717/492466 [1:26:53<13:06, 83.62it/s]

In [24]:
df.to_parquet("./700077347251945472_sentiment.parquet")